# Example Simple RAG

In [27]:
!pip install ragatouille
!pip install chromadb
!pip install langchain langchain-ollama langchain-community langchain-huggingface --upgrade 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.4 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.3
    Uninstalling langchain-0.3.3:
      Successfully uninstalled langchain-0.3.3
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.2
    Uninstalling langchain-community-0.3.2:
      Successfully uninstalled langchain-community-0.3.2


In [31]:
import chromadb
import pandas as pd
import sqlite3
from ragatouille import RAGPretrainedModel
import ollama
from typing import Optional
import langchain
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
from langchain_ollama.llms import OllamaLLM
import os

## Define the Corpus

In [53]:
corpus = pd.read_csv('../data_raw/WMT_Grocery_202209.csv')
corpus

/tmp/ipykernel_23459/3028156575.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  corpus = pd.read_csv('../data_raw/WMT_Grocery_202209.csv')


,SHIPPING_LOCATION,DEPARTMENT,CATEGORY,SUBCATEGORY,BREADCRUMBS,SKU,PRODUCT_URL,PRODUCT_NAME,BRAND,PRICE_RETAIL,PRICE_CURRENT,PRODUCT_SIZE,PROMOTION,RunDate,tid
0,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",110895339,https://www.walmart.com/ip/Marketside-Roasted-...,"Marketside Roasted Red Pepper Hummus, 10 Oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163804
1,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",105455228,https://www.walmart.com/ip/Marketside-Roasted-...,"Marketside Roasted Garlic Hummus, 10 Oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163805
2,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",128642379,https://www.walmart.com/ip/Marketside-Classic-...,"Marketside Classic Hummus, 10 Oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163806
3,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",366126367,https://www.walmart.com/ip/Marketside-Everythi...,"Marketside Everything Hummus, 10 oz",Marketside,2.67,2.67,10,NaN,2022-09-11 21:20:04,16163807
4,79936,Deli,"Hummus, Dips, & Salsa",NaN,"Deli/Hummus, Dips, & Salsa",160090316,https://www.walmart.com/ip/Price-s-Jalapeno-Di...,"Price's Jalapeno Dip, 12 Oz.",Price's,3.12,3.12,12,NaN,2022-09-11 21:20:04,16163808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568529,70072,Alcohol,Wine,White Wine,Alcohol/Wine,593600139,https://www.walmart.com/ip/Farm-Fresh-Blueberr...,Farm Fresh Blueberry Moscato 750ml,Farm Fresh Wine Company,9.98,9.98,750,NaN,2022-09-11 21:20:04,16732333
568530,70072,Alcohol,Wine,White Wine,Alcohol/Wine,333403243,https://www.walmart.com/ip/Farm-Fresh-Peach-Mo...,Farm Fresh Peach Moscato 750 Ml,Farm Fresh Wine Company,9.98,9.98,750,NaN,2022-09-11 21:20:04,16732334
568531,70072,Alcohol,Wine,White Wine,Alcohol/Wine,526588325,https://www.walmart.com/ip/Farm-Fresh-Raspberr...,Farm Fresh Raspberry Moscato 750ml,Farm Fresh Wine Company,9.98,9.98,750,NaN,2022-09-11 21:20:04,16732335
568532,70072,Alcohol,Wine,White Wine,Alcohol/Wine,286992782,https://www.walmart.com/ip/Farm-Fresh-Mango-Mo...,Farm Fresh Mango Moscato 750ml,Farm Fresh Wine Company,9.98,9.98,750,NaN,2022-09-11 21:20:04,16732336


In [54]:
corpus['CATEGORY'].isna().sum()

0

## Preprocess and Load the Corpus into the Vector Store

In [15]:
sqlcon = sqlite3.connect("./sqllite/walmart.db")

In [16]:
os.chmod("./sqllite/walmart.db", 774)
corpus.to_sql("WalMart", sqlcon)

568534

In [32]:
os.chmod("./sqllite/walmart.db", 444)

In [18]:
cursor = sqlcon.cursor()
cursor.execute("SELECT PRICE_RETAIL FROM WalMart WHERE CATEGORY='Beer';")
print(cursor.fetchall())

[(20.98,), (16.48,), (16.48,), (22.98,), (20.98,), (21.98,), (18.98,), (20.98,), (22.48,), (12.98,), (14.98,), (20.98,), (6.98,), (18.98,), (16.98,), (20.98,), (20.98,), (18.73,), (21.48,), (21.48,), (21.98,), (22.48,), (15.98,), (12.98,), (20.98,), (17.48,), (14.98,), (12.98,), (16.98,), (12.98,), (12.48,), (7.98,), (12.98,), (9.98,), (16.98,), (14.98,), (6.98,), (12.98,), (12.48,), (7.98,), (10.98,), (13.48,), (7.48,), (13.48,), (1.98,), (22.48,), (6.48,), (12.48,), (12.48,), (14.98,), (6.98,), (10.98,), (12.48,), (15.98,), (9.48,), (20.48,), (20.98,), (2.48,), (12.48,), (9.48,), (6.98,), (15.98,), (10.48,), (13.48,), (2.48,), (17.27,), (17.98,), (2.98,), (6.48,), (9.98,), (2.48,), (10.98,), (16.48,), (7.48,), (14.98,), (14.98,), (9.48,), (11.48,), (13.48,), (7.48,), (17.98,), (9.98,), (7.98,), (2.48,), (9.48,), (18.98,), (14.48,), (12.48,), (2.48,), (5.98,), (2.98,), (16.48,), (1.98,), (16.48,), (19.98,), (19.98,), (16.98,), (16.98,), (8.48,), (19.98,), (16.98,), (16.48,), (8.48,), 

In [33]:
db = SQLDatabase.from_uri("sqlite:///sqllite/walmart.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT PRICE_RETAIL FROM WalMart WHERE CATEGORY='Beer' LIMIT 10;")

sqlite
['WalMart']


'[(20.98,), (16.48,), (16.48,), (22.98,), (20.98,), (21.98,), (18.98,), (20.98,), (22.48,), (12.98,)]'

In [3]:
chunks = {}
for category in corpus['CATEGORY'].unique():
    chunks[category] = []
print(chunks)

{'Hummus, Dips, & Salsa': [], 'Energy Drinks': [], 'Oils & Shortening': [], 'Fresh Soups & Salads': [], 'Game Time Faves': [], 'Sparkling Water': [], 'Better for you': [], 'Healthy Snacks': [], 'Pudding & Gelatin': [], 'Pretzels': [], 'Granola Bars': [], 'Baking Nuts & Seeds': [], 'Yeast': [], 'Milk': [], 'Cheese': [], 'Halloween candy': [], 'Canned & Powdered Milks': [], 'Specialty Cheeses & Meats': [], 'Bacon, Hot Dogs, Sausage': [], 'Variety Pack Snacks': [], 'Drink Mixes': [], 'Eggs': [], 'Orange Juice & Chilled': [], 'Cream & Creamers': [], 'Beverage Deals': [], 'Hard candy & lollipops': [], 'Meat Sticks': [], 'Fruit Snacks': [], 'Chocolate': [], 'Fresh Pizza': [], 'Sugars & Sweeteners': [], 'New Arrivals': [], 'Biscuits, Cookies, Doughs': [], 'Top baking brands': [], 'Mints': [], 'Prepared Meals & Sides': [], 'Yogurt': [], 'Popcorn': [], 'Chips': [], 'Rotisserie Chicken': [], 'Juices': [], 'Deli Meat & Cheese': [], 'Gummy & chewy candy': [], 'Non-Alcoholic Mixers': [], 'Butter & 

In [69]:
len(corpus.groupby('BRAND')['CATEGORY'])

4368

In [70]:
chunks = {}
for category in corpus['CATEGORY'].unique():
    chunks[category] = {}
print(chunks)

{'Hummus, Dips, & Salsa': {}, 'Energy Drinks': {}, 'Oils & Shortening': {}, 'Fresh Soups & Salads': {}, 'Game Time Faves': {}, 'Sparkling Water': {}, 'Better for you': {}, 'Healthy Snacks': {}, 'Pudding & Gelatin': {}, 'Pretzels': {}, 'Granola Bars': {}, 'Baking Nuts & Seeds': {}, 'Yeast': {}, 'Milk': {}, 'Cheese': {}, 'Halloween candy': {}, 'Canned & Powdered Milks': {}, 'Specialty Cheeses & Meats': {}, 'Bacon, Hot Dogs, Sausage': {}, 'Variety Pack Snacks': {}, 'Drink Mixes': {}, 'Eggs': {}, 'Orange Juice & Chilled': {}, 'Cream & Creamers': {}, 'Beverage Deals': {}, 'Hard candy & lollipops': {}, 'Meat Sticks': {}, 'Fruit Snacks': {}, 'Chocolate': {}, 'Fresh Pizza': {}, 'Sugars & Sweeteners': {}, 'New Arrivals': {}, 'Biscuits, Cookies, Doughs': {}, 'Top baking brands': {}, 'Mints': {}, 'Prepared Meals & Sides': {}, 'Yogurt': {}, 'Popcorn': {}, 'Chips': {}, 'Rotisserie Chicken': {}, 'Juices': {}, 'Deli Meat & Cheese': {}, 'Gummy & chewy candy': {}, 'Non-Alcoholic Mixers': {}, 'Butter & 

In [73]:
for row in range(len(corpus)):
    category = corpus['CATEGORY'][row]
    brand = corpus['BRAND'][row]
    if brand not in chunks[category]:
        chunks[category][brand] = [corpus.loc[row]]
    else:
        chunks[category][brand].append([corpus.loc[row]])

In [78]:
print(len(chunks))
print(len(chunks['Frozen Desserts']['Great Value']))

114
1388


In [105]:
chunks['Frozen Desserts']['Great Value'][0].index

Index(['SHIPPING_LOCATION', 'DEPARTMENT', 'CATEGORY', 'SUBCATEGORY',
       'BREADCRUMBS', 'SKU', 'PRODUCT_URL', 'PRODUCT_NAME', 'BRAND',
       'PRICE_RETAIL', 'PRICE_CURRENT', 'PRODUCT_SIZE', 'PROMOTION', 'RunDate',
       'tid'],
      dtype='object')

In [108]:
ids = []
metadata = []
docs = []
i = 0
for category in chunks:
    for brand in chunks[category]:
        temp_docs = []
        ids.append(f"{i+1}")
        i+=1
        metadata.append({'category':category, 'brand':brand})
        for doc in range(len(chunks[category][brand])):
            for string in range(len(chunks[category][brand][doc])):
                temp_docs.append(chunks[category][brand][0].index[string])
                temp_docs.append(chunks[category][brand][doc][string])
        docs.append(" ".join(str(x) for x in temp_docs))

In [109]:
print(len(ids))
print(len(docs))
print(docs[-1])
print(metadata[-1])

9865
9865
SHIPPING_LOCATION 33647 DEPARTMENT Alcohol CATEGORY Beer SUBCATEGORY Imported Beer BREADCRUMBS Alcohol/Beer SKU 133324633 PRODUCT_URL https://www.walmart.com/ip/Presidente-Pilsener-Style-Beer-12-Pack-12-fl-oz-Bottles-5-ABV/133324633?fulfillmentIntent=Pickup PRODUCT_NAME Presidente Pilsener Style Beer, 12 Pack 12 fl. oz. Bottles, 5% ABV BRAND Presidente PRICE_RETAIL 15.73 PRICE_CURRENT 15.73 PRODUCT_SIZE 12 PROMOTION nan RunDate 2022-09-11 21:20:04 tid 16725918
{'category': 'Beer', 'brand': 'Presidente'}


In [110]:
chroma_client = chromadb.PersistentClient(path="./split_chroma")
#chroma_client.delete_collection(name="ghibli") # In case we need to redo anything
collection = chroma_client.get_or_create_collection(name="grocery")

In [34]:
chroma_client = chromadb.PersistentClient(path="./newer_chroma")
#chroma_client.delete_collection(name="ghibli") # In case we need to redo anything
collection = chroma_client.get_or_create_collection(name="grocery")

In [2]:
chroma_client = chromadb.PersistentClient(path="./chroma")
#chroma_client.delete_collection(name="ghibli") # In case we need to redo anything
collection = chroma_client.get_or_create_collection(name="grocery")

In [111]:
# Take the lists and load them into chromadb

batch_size = 10000
batch_count = (len(docs) - 1) // batch_size + 1

for batch in range(batch_count):
    db_docs = docs[batch*batch_size:(batch+1)*batch_size]
    db_ids = ids[batch*batch_size:(batch+1)*batch_size]
    db_metadata = metadata[batch*batch_size:(batch+1)*batch_size]
    print(f"Adding batch {batch}/{batch_count} to vector store")
    collection.add(documents=db_docs, ids=db_ids, metadatas=db_metadata)

Adding batch 0/1 to vector store


In [122]:
# Display an example search against chromadb

results = collection.query(
    query_texts=["How much does shrimp cost?"],
    n_results=1
)
print(results)

{'ids': [['4400']], 'distances': [[0.9189187288284302]], 'metadatas': [[{'brand': 'Pacific Seafood', 'category': 'Fresh Food'}]], 'embeddings': None, 'documents': [['SHIPPING_LOCATION 85225 DEPARTMENT Meat & Seafood CATEGORY Fresh Food SUBCATEGORY Seafood BREADCRUMBS Meat & Seafood/Fresh Food SKU 210567362 PRODUCT_URL https://www.walmart.com/ip/Frozen-Wild-Coldwater-Salad-Shrimp-1-lb-350-500-Count-per-lb/210567362?fulfillmentIntent=Pickup PRODUCT_NAME Frozen Wild Coldwater Salad Shrimp, 1 lb (350-500 Count per lb) BRAND Pacific Seafood PRICE_RETAIL 8.36 PRICE_CURRENT 8.36 PRODUCT_SIZE 1 PROMOTION nan RunDate 2022-09-11 21:20:04 tid 16489619']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [47]:
model = OllamaLLM(model="llama3.1")
chain = create_sql_query_chain(model, db)
response = chain.invoke({"question": "How much does beer cost"})
response

'However, I need to inform you that there is no column for "beer" in the provided tables.\n\nBut I can help you with a query that will return all products with their prices. Then we can filter the results to find the price of beer if it was present.\n\nSQLQuery: SELECT `"PRICE_CURRENT"` FROM `"WalMart'

In [7]:
# Define the reranker

llmreranker = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/home/mist861/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/mist861/anaconda3/lib/python3.11/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [118]:
# The below takes the prompt, performs a retrieval against chromadb, reranks it, then calls ollama. I did not make (most) of this, it's derived from: https://huggingface.co/learn/cookbook/en/advanced_rag

def rag_query(
    question: str,
    llm: str,
    knowledge_index=collection,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 5,
    num_docs_final: int = 1):
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.query(query_texts=question, n_results=num_retrieved_docs)
    relevant_docs = relevant_docs['documents'][0]

    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)

    relevant_docs = relevant_docs[:num_docs_final]

    final_prompt = f"""
        You are a helpful assistant. Use the provided context to answer the provided question. 
        The attribute names of each item are set in UPPERCASE followed by the attribute value.
        For example, the cost or price of an item follows the term "PRICE_CURRENT".
        
        CONTEXT: {relevant_docs}
        QUESTION: {question}
        
        """

    response = ollama.chat(model=llm, messages=[
        {
            'role': 'user',
            'content': final_prompt,
        },
    ])
    answer = response['message']['content']

    return answer, relevant_docs

In [123]:
# Perform a retrieval/generations without the reranker

question="How much does shrimp cost?"
result, docs = rag_query(question, 'llama3.1')
print(result)

=> Retrieving documents...
According to the provided context, the current price of the shrimp is $8.36, which is indicated by the attribute name "PRICE_CURRENT".


In [50]:
# Display the retrieved documents used

print(docs)
len(docs)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



5

In [ ]:
# Perform a retrieval/generation with the reranker

result, docs = rag_query(question, 'llama3.1', reranker=llmreranker)

=> Retrieving documents...
=> Reranking documents...


/home/mist861/anaconda3/lib/python3.11/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


In [16]:
print(result)

I don't know. There is no mention of hummus in the provided context.


In [17]:
print(docs)

[{'content': '60007 Deli Deli Meat & Cheese nan Deli/Deli Meat & Cheese 483646515 https://www.walmart.com/ip/Great-Value-Original-Pepperoni-Mega-Pack-21-oz/483646515?fulfillmentIntent=Pickup Great Value Original Pepperoni Mega Pack, 21 oz Great Value 7.68 7.68 21 nan 2022-09-11 21:20:04 16174372', 'score': 7.5078125, 'rank': 0, 'result_index': 14}, {'content': '48180 Deli Deli Meat & Cheese nan Deli/Deli Meat & Cheese 433305422 https://www.walmart.com/ip/Great-Value-Honey-Ham-Mega-Pack-lunchmeat-22-ounce/433305422?fulfillmentIntent=Pickup Great Value Honey  Ham Mega Pack lunchmeat, 22 ounce Great Value 6.98 6.98 22 nan 2022-09-11 21:20:04 16253260', 'score': 7.4375, 'rank': 1, 'result_index': 4}, {'content': '48180 Deli Deli Meat & Cheese nan Deli/Deli Meat & Cheese 483646515 https://www.walmart.com/ip/Great-Value-Original-Pepperoni-Mega-Pack-21-oz/483646515?fulfillmentIntent=Pickup Great Value Original Pepperoni Mega Pack, 21 oz Great Value 7.68 7.68 21 nan 2022-09-11 21:20:04 1625328